# Creating an agent with MCP

The [Model Context Protocol](https://www.anthropic.com/news/model-context-protocol) (MCP) introduced by Anthropic has proven to be a popular method for providing an AI agent with access to a variety of tools. [This Huggingface blog post ](https://huggingface.co/blog/Kseniase/mcp) has a nice explanation of MCP.  In this tutorial, we'll build an agent that is able to leverage MCP server provided tools.

Note: because this tutorial relies upon advanced stdio/stderr communication using the MCP Server, it cannot be run on Google Colab.

## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks, this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio.

In [1]:
%pip install 'any-agent'

import nest_asyncio

nest_asyncio.apply()

Note: you may need to restart the kernel to use updated packages.


In [5]:
from dotenv import load_dotenv
import google.generativeai as genai
import os

load_dotenv()  # Busca automáticamente un archivo .env
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel('gemini-2.5-flash')
response = model.generate_content("Explain how AI works in a few words")
print(response.text)

/Users/ANDRES/Documents/GitHub/ai-agent-htw/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AI learns patterns from data to make predictions or decisions.


## Configure the Agent

Now it's time to configure the agent! At this stage you have a few choices:

### Pick the framework

We support a variety of underlying agent frameworks (OpenAI, Smolagents, Langchain, TinyAgent, etc), which all have their own particular agentic AI implementations. For this tutorial's simple use case, any of the frameworks should work just fine, but any-agent makes it easy to try out a different framework later, if we so choose. For this example, we will use the [TinyAgent](frameworks/tinyagent.md) framework.  

### Pick an LLM

Regardless of which agent framework you choose, each framework supports LiteLLM, which is a proxy that allows us to use whichever LLM inside the framework, hosted on by any provider. For example, we could use a local model via llama.cpp or llamafile, a google hosted gemini model, or a AWS bedrock hosted Llama model. For this example, let's use Mistral AI's mistral/mistral-small-latest.

### Pick which tools to use

 In this example, we'll add a few MCP servers that we host locally, which means we'll use a Stdio MCP server. If an MCP Server is already running and hosted elsewhere, you can use an SSE connection to access it. You can browse some of the officially supported MCP servers [here](https://github.com/modelcontextprotocol/servers/tree/main?tab=readme-ov-file).

 Lets give use two MCP servers: 
 
 * [Time](https://github.com/modelcontextprotocol/servers/tree/main/src/time): so the agent can know what time/day it is.
 * [Airbnb](https://github.com/openbnb-org/mcp-server-airbnb): so the agent can browse airbnb listings

 I will also add a custom send_message tool, that way it can ask us additional questions before getting its final answer!
  

In [2]:
import os
from getpass import getpass

if "GEMINI_API_KEY" not in os.environ:
    print("GEMINI_API_KEY not found in environment!")
    api_key = getpass("Please enter your GEMINI_API_KEY: ")
    os.environ["GEMINI_API_KEY"] = api_key
    print("GEMINI_API_KEY set for this session!")
else:
    print("GEMINI_API_KEY found in environment.")

GEMINI_API_KEY found in environment.


In [7]:
!brew install node

==> Downloading https://formulae.brew.sh/api/formula.jws.json
==> Downloading https://formulae.brew.sh/api/cask.jws.json
==> Fetching downloads for: node
==> Downloading https://ghcr.io/v2/homebrew/core/node/manifests/24.4.1-2
######################################################################### 100.0%
==> Fetching dependencies for node: c-ares, libnghttp3, ca-certificates, openssl@3, libngtcp2, libuv, simdjson, readline and sqlite
==> Downloading https://ghcr.io/v2/homebrew/core/c-ares/manifests/1.34.5-1
######################################################################### 100.0%
==> Fetching c-ares
==> Downloading https://ghcr.io/v2/homebrew/core/c-ares/blobs/sha256:76858a7f6dd
######################################################################### 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/libnghttp3/manifests/1.10.1-1
######################################################################### 100.0%
==> Fetching libnghttp3
==> Downloading https://ghcr.io/v2/hom

In [8]:
from any_agent import AgentConfig, AnyAgent
from any_agent.config import MCPStdio

# This MCP Tool relies upon uvx https://docs.astral.sh/uv/getting-started/installation/
time_tool = MCPStdio(
    command="uvx",
    args=["mcp-server-time", "--local-timezone=America/New_York"],
    tools=[
        "get_current_time",
    ],
    client_session_timeout_seconds=30,
)

# This MCP tool relies upon npx https://docs.npmjs.com/cli/v8/commands/npx which comes standard with npm
airbnb_tool = MCPStdio(
    command="npx",
    args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
    client_session_timeout_seconds=30,
)


# This is a custom tool that we will provide to the agent. For the agent to use the tool, we must provide a docstring
# and also have proper python typing for input and output parameters
def send_message(message: str) -> str:
    """Display a message to the user and wait for their response.

    Args:
        message: str
            The message to be displayed to the user.

    Returns:
        str: The response from the user.

    """
    if os.environ.get("IN_PYTEST") == "1":
        return "2 people, next weekend, low budget. Do not ask for any more information or confirmation."
    return input(message + " ")


agent = AnyAgent.create(
    "tinyagent",  # See all options in https://mozilla-ai.github.io/any-agent/
    AgentConfig(
        model_id="gemini/gemini-2.5-flash",
        tools=[airbnb_tool, time_tool, send_message],
    ),
)

## Run the Agent

Now we've configured our agent, so it's time to run it! Since it has access to airbnb listings as well as the current time, it's a perfect fit for helping me find a nice airbnb for the weekend.


In [9]:
prompt = """
I am looking to book an airbnb next weekend near Ohiopyle, PA. Can you help me plan this? Figure out the time, then ask me some questions and lets figure this out together. Once I tell you what you need to know, find and return some options for me.
"""

agent_trace = agent.run(prompt)

╭─────────────────────────────────────── CALL_LLM: gemini/gemini-2.5-flash ───────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "system",                                                                                       │ │
│ │     "content": "You are an agent - please keep going until the user's query is completely resolved, before  │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "\nI am looking to book an airbnb next weekend near Ohiopyle, PA. Can you help me plan this? │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "get_current_time",                                                                        │ │
│ │     "tool.args": "{\"timezone\": \"America/New_York\"}"                                                     │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 876,                                                                                      │ │
│ │   "output_tokens": 165,                                                                                     │ │
│ │   "input_cost": 0.0002628,                                                                                  │ │
│ │   "output_cost": 0.00041250000000000005                                                                     │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── EXECUTE_TOOL: get_current_time ─────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "timezone": "America/New_York"                                                                            │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Error calling tool get_current_time: Timed out while waiting for response to ClientRequest. Waited 30.0     │ │
│ │ seconds.                                                                                                    │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────── CALL_LLM: gemini/gemini-2.5-flash ───────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ I can help you with this! Unfortunately, my tool to get the current time timed out, so I cannot determine   │ │
│ │ the exact dates for "next weekend" automatically.                                                           │ │
│ │                                                                                                             │ │
│ │ Could you please tell me the specific check-in and check-out dates you have in mind for your trip to        │ │
│ │ Ohiopyle, PA?                                                                                               │ │
│ │                                                                                                             │ │
│ │ Also, to help me find the best options, please tell me:                                                     │ │
│ │                                                                                                             │ │
│ │  • How many adults will be staying?                                                                         │ │
│ │  • Will there be any children or infants? If so, how many of each?                                          │ │
│ │  • Will you be bringing any pets? If so, how many?                                                          │ │
│ │  • Do you have a preferred price range (minimum and maximum) per night?                                     │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 941,                                                                                      │ │
│ │   "output_tokens": 285,                                                                                     │ │
│ │   "input_cost": 0.0002823,                                                                                  │ │
│ │   "output_cost": 0.0007125                                                                                  │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## View the results 

The `agent.run` method returns an AgentTrace object, which has a few convenient attributes for displaying some interesting information about the run.

In [10]:
print(agent_trace.final_output)  # Final answer
print(f"Duration: {agent_trace.duration.total_seconds():.2f} seconds")
print(f"Total Tokens: {agent_trace.tokens.total_tokens:,}")
print(f"Total Cost (USD): {agent_trace.cost.total_cost:.6f}")

I can help you with this! Unfortunately, my tool to get the current time timed out, so I cannot determine the exact dates for "next weekend" automatically.

Could you please tell me the specific check-in and check-out dates you have in mind for your trip to Ohiopyle, PA?

Also, to help me find the best options, please tell me:
*   How many adults will be staying?
*   Will there be any children or infants? If so, how many of each?
*   Will you be bringing any pets? If so, how many?
*   Do you have a preferred price range (minimum and maximum) per night?
Duration: 33.90 seconds
Total Tokens: 2,267
Total Cost (USD): 0.001670
